In [1]:
from itertools import combinations
import re
from tqdm import tqdm
from functools import lru_cache

In [2]:
def generate_choices(length: int, num_springs: int):
    indices = list(range(length))
    for combo in combinations(indices, num_springs):
        pattern = ["." for _ in range(length)]
        for i in combo:
            pattern[i] = "#"
        yield pattern
        
def hide_status(springs_str: str, unknown_indices: list[int]) -> str:
    out = list(springs_str)
    for ind in unknown_indices:
        out[ind] = '?'
    return "".join(out)

def get_choices(unknown_indices, springs, conditions) -> list[str]:
    out = []
    # generate reasonable choices
    for choice in generate_choices(
        length=len(springs), num_springs=sum(conditions)
    ):
        choice_str = "".join(choice)
        # check the resulting condition is matching the spring condition
        choice_condition = [len(t) for t in re.findall(r"\#+", choice_str)]
        if choice_condition != conditions:
            continue
        # check the str is the same
        if hide_status(choice_str, unknown_indices) != springs:
            continue
        # good choice option to append to list
        out.append(choice_str)  
    return out

In [8]:
class SpringRowCondition:
    def __init__(self, springs: str, conditions: list[int]):
        self.springs = springs
        self.conditions = conditions
        
    def __repr__(self) -> str:
        return f"SpringRowCondition(spring={self.springs}, conditions={self.conditions})"

    @classmethod
    def from_str(cls, value: str, part_2: bool = False) -> 'SpringRowCondition':
        springs_str, condition_str = value.split(' ')
        if part_2:
            springs_str = '?'.join([springs_str for _ in range(5)])
            springs_str = ''.join(springs_str)
            condition_str = ",".join([condition_str for _ in range(5)])
            
        return cls(springs=springs_str, conditions=[int(t) for t in condition_str.split(',')])
    
    @property
    def unknown_indices(self) -> list[int]:
        return [i for i, t in enumerate(self.springs) if t =='?']

    
class Game:
    def __init__(self, field: list[SpringRowCondition]) -> None:
        self.field = field
        
    @classmethod
    def from_str(cls, value: str, part_2: bool = False):
        return cls(field=[SpringRowCondition.from_str(t, part_2=part_2)for t in value.strip().split('\n')])
    
    def get_from_choice(self, conditions: list[int]) -> list[SpringRowCondition]:
        return [t for t in self.field if t.conditions==conditions]

    def __repr__(self) -> str:
        return f"Game(field={self.field})"
    
    def solution_part_1(self):
        result = 0
        for row in tqdm(self.field):
            choices = get_choices(row.unknown_indices, row.springs, row.conditions)
            result += len(choices)
        return result

In [14]:
puzzle_str = """
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
"""

# Puzzle

In [10]:
puzzle_str = """    
..?##?.?.??#???#? 3,7
???.##??##???#??.# 1,1,7,1,1,1
?#.??.??#.??#? 1,1,2,2
?????.?#???? 1,2,1,1
??##???.?.???. 6,1,2
#??#.??.?.????.#?? 1,1,1,1,3,1
??.??????.?#? 2,1,3,1
??##?.??#? 4,3
?#?.???#?????# 2,4,1,1
#?#??.??##?#?#?#???? 3,13
??????##??? 1,4
??#????????#??. 7,1
?????##???. 1,2,1
.??..???????## 1,1,7
??#?#?.#??.? 1,1,1,1
?.???????? 3,1
??.??.#?#?#?###? 2,1,9
???.?????? 3,1,1
?#?#?#???.?#?.??#. 6,2,2,3
?.??.#.??????? 1,1,3,1
??#???..??#?.?#. 1,3,1,1,2
??#?#???##?#?#??. 2,6,1,1,1
???????#??.??. 1,2
##??????????#?.????# 9,2,1,2
#?#.#.????#???. 1,1,1,3,1
?#????.???? 3,1
??#??#?????#.??????? 9,1,2,3
#?.?#???#?##???.#.? 1,10,1,1
?..??.?????#??#?. 1,7
?.?#????#?.? 2,4
.?#????.???? 2,1,1,2
??#??.?#???.#?# 3,1,4,3
??.?.??#???? 2,5
??..###.?????#.? 1,3,1,1,1
..??????#??? 1,7
?.?#???#.???# 1,5,4
.??.?.#?#???#? 1,5,1
?#?#??..??#. 1,1,1,3
..??##???????? 7,3
?#??#???#.??.?????#. 9,2,2,1
?.??????..???#??# 1,1,3,7
#?#??.?#?#?#??. 5,5
??.?.?#.??#? 2,1,2
???.????????#? 1,1,1,3
?.#.?#??#?#???. 1,1,7,2
.??#.#?#???#???. 3,5,2,1
???.?#??#???? 2,6
???##???#?????. 1,7,1,1
??????#??#?###?..??? 1,11
???????#.? 1,4
.?????.??. 1,1,1
?#???.?..??##?#?.? 1,1,1,7
????.??#??.? 1,1,4
#..?.??.??? 1,1,3
.?#????..??#?? 6,3
??.??????#?? 1,5
????#???.? 1,6
#???#???#. 1,3,1
.??????.???##?. 2,6
??#?????????.?? 1,2,2,2,1
?.????.??.##??? 2,3
???#???.???.?.#. 5,1,1,1,1
??#??#???.#? 6,1
?..#.??###?#???? 1,9
..??.?#??.????#? 2,1,1,3,1
.???#?##?????? 1,6,2
.#????.??? 1,3
???#????#???. 2,6
??#???#????#?..?#?#? 2,3,4,3
.??.?????#?????###?# 1,1,1,3,6
?.??#?..????? 3,4
??#..???.? 2,1
?###???#???#? 5,4,1
.?##?##?.?###. 6,3
?????.#????. 1,1,1,5
..???????#.???#. 1,3,1,2
???#?..??#????.???? 1,3,6,1
.#?????#?? 1,5
?????.????#?? 2,1,1,3
???.???.??#?#?? 2,1,1,6
??????#???.#?????# 2,3,2,1,3,1
??#?#?#?????#?.# 1,1,8,1
???###??.#?? 1,4,1,1
?#???..?..?.? 5,1
??.?.???#?#. 1,1,3,1
.##??#?#?.????# 2,5,3
#.??#?.??## 1,3,4
..??..?.??.#.?#??#? 2,1,2,1,4
.?#??#?#?#?? 2,6,1
.?##.?.?#? 3,1
?????#?..#??.##? 7,1,1,2
?????..?????#???? 3,3,2
.????.??#?? 3,2
????.?.????? 1,1,1,1
?#.???.?#?.??.#? 2,1,3,1,1
.#????##???# 2,4,1
????#.?.?#?#? 1,1,5
??????????# 4,1,2
.?.?#????#?#??. 1,6,2
???#???#???# 1,3,1
#??#??#??? 2,4,1
???.?##?#?.?..??.. 1,1,2,1,1,2
.?.#??##?##? 1,2,6
??#.?.?#???#.?# 1,1,5,2
.??..????#??#??.??? 2,3,1,2,3
?.#????#???.. 1,5
?#?##??..?????#. 5,1,3,1
??????.?#??#???. 4,1,7
????##??.##??#?##?? 3,9
?#??.??????? 3,1
?.?????#?#???? 5,3
?..???.#??? 2,3
????#???#.??#?# 1,5,2,1
?###??#???#??.?#??? 12,2
??#??#????????#?. 5,7
????#.##???? 1,1,2,1
...???????????#??.? 2,1,4
??????????##??? 1,11
#..???.?#? 1,1,2
?#?????#?????? 4,2,2,1
??????.###???? 1,3,4,1
.?.?????#??#?. 1,5,2
???.??.?.. 1,1
???????#..??#?#?? 2,2,1,5,1
.#????#??? 1,1,1
.???.????#??? 1,3
.???????#??? 1,6,1
.?.?????#??#? 2,3,1
???????????.?.?# 9,1
#????..?..#?#?# 4,1,1,3
??#?.?#?#?#??#?????? 1,13
??#.?.????#?# 1,2,1,1
#???.????##??. 2,1,1,4
.####?????#.???##??? 10,2,3
..??.???.? 1,1,1
???????.#??#? 5,1,2,1
..?##??#?? 3,1
??????.?..##?####?.? 4,8
#??.??????#???#????. 3,2,1,1,3,2
#??#?#????.???. 1,3,1,1,3
.???.??.??#. 3,2
#??#?#?#.??.?????.? 6,1,1,1,1,1
.???#??.?#? 4,2
?#.??##????.# 1,6,1,1
.???####????.?.????? 11,3
.?.?#??..???##???#. 1,3,8
?????.??????#?.?# 2,7,2
??.#.#????#? 1,1,6
..??.#??????? 1,2,1,1
??.??#??#?. 2,1,3
.??????.?.? 2,1,1
????##?????? 1,3,2
???##.?#?#??#?#.?.? 2,2,3,1,1
???.?#?..? 1,2,1
#??????#?#???.?? 1,1,2,2,1
.##..??.#???.#? 2,1,4,1
.????????#??#?#???? 6,8
???#????#??? 1,3,1
?#???#????? 5,2
???#.??????????##? 4,4,6
?##??.???#.??#? 4,1,3
????????#?.#???.. 6,4
????#.?.???#?####? 5,1,1,7
????????#??#??#?. 1,9
??.????????#??.????? 2,1,6,2,1
???.????????????#.? 2,1,4
???#??#???#.?#??##.? 7,1,1,2,2,1
.?#.??##.??.??? 1,3,1,1,1
?.?#.?#?.?.#? 2,1,1,1
????#????#???.??.? 1,3,6,1
##.??##.?? 2,3
?????.???#. 2,1,3
??...#.?????##?? 2,1,9
????.?.?#. 3,1,1
??.#??#?#?.?? 1,6,1
???##??#?#???? 3,7
???#????#?#? 2,4,3
??????.?#?#?.?? 4,5
??##?#?????#. 6,2
.???###?????#?#??? 12,4
#.???.?????.?#?.? 1,1,1,2,3
??#???????????#? 6,1,1,1
.???#???#?##???. 2,9
..#????###??..#??# 1,1,5,2,1
#?????#??#?##????..? 5,11,1
.???????.?. 1,1,1
??...???????????? 4,1
#??##??#??.?##? 8,1,3
?..??.??????.? 1,1,6
.??#?.?#???? 1,2,1,2
??????.????? 1,1,1
?#???###?.#. 2,5,1
#???.#??#?.##???? 1,1,5,3,1
?###??##???.??#????? 9,5,1
?##.??##?.?. 3,3,1
?.???.#??????##.?#? 1,1,1,9,1
..?#.?.????##?? 2,3,2
?#?.?##.???? 1,2,1,1
.#?????#?##### 1,1,9
??#..???#?#????????? 1,6
#.??????##??#?#.??#? 1,5,7,1,1
.?.?#????#???????? 1,3,2,1,3
.#???.????# 4,1,2
#???##??#????. 6,1,1
?.???????.???? 6,1
??#?#??#?..??????. 4,3,2,3
???#?.??.? 5,1,1
??.?#????? 1,1,1
??????????#??.?.?# 1,1,1,6,1,1
.??#?#???#??# 1,1,2,5
?#??##.??????? 5,6
??.??.?????#?#????? 1,1,10
..?????#.?.?..???? 6,2
?##??#.???.#?#???? 6,2,3,2
#?#?#?..??? 3,2,1
.#?#?##???.## 7,2
#???..???#?#?????# 4,1,9
?.?#????##?.???? 9,1,1
.????????? 2,2
#????????????. 1,2,2,1
??.???#.#?? 1,2,2
?????????? 1,1,3
.?.?????#?? 1,2
??#???.???#?##??#? 5,1,6,1
.#???.???? 4,1,1
.#??###???.????##??? 7,5
#???#??#????.?. 1,4,1
???##?##??????#?? 1,6,6
???#?????#????#? 5,9
.?.?..?.???.#???. 1,3
??#?.???.????? 2,3
.#?.##??#.?? 2,2,2,1
#.??????????.?.? 1,4,2,1,1
?.?####??????#.??? 11,1
.????.????? 1,2,1
??#.?????##?##? 2,7,2
?.??????????#?# 1,1,2,1,3
????#.?#?###. 1,6
???????#????#?? 1,1,4,2
.?#?#?#?????#. 9,2
?.????#??.? 4,1
?#?????.???#???.??? 1,2,1,3,1,3
?#???????#?.#.????? 1,7,1,1,3
.#??.???.??#??????# 3,3,9
#????..???#..#? 5,3,2
?.?.#?????.?? 1,6,1
.??..#??#?. 1,1,3
??###???#????##??? 10,3
??#??#?#?#?..??#???? 9,4
???#???????#? 4,6
?.#.?##?.??????##?? 1,4,6
??#???.?#??#? 2,1,6
??????##??.????? 8,1,1
?????#..?. 5,1
##.???.?...?? 2,1,1,1
????????##?#?# 1,7,1,1
??????????????? 8,1,1
???.??###? 1,1,5
????????????????##.? 1,4,1,3,3,1
??????????#?#? 1,3,4
.????.?.???#.???#?.? 4,1,4,2,2,1
.??#?#??#. 1,3,1
??.##.???#?#?.?#. 1,2,2,1,2
???#????#???.???#?? 4,2,1,3
.???.#?????..?? 1,6,1
..#?#????.. 1,2,1
??#?.?##?##? 4,5
?#?.?.???? 2,1
???????.#??#??#? 1,1,3,1,5
??.?.#.?????#??? 1,1,1,4
??#?.?#?????#?.??? 2,4,1,2
??????.??##?#??#?. 2,10
?????????#? 2,1,4
..??#????? 3,2
?????????###?? 1,7
#.?????????..????## 1,3,1,1,1,4
???#??.#.?.??? 1,3,1,1
??????#?#.#???#? 8,1,1
?#??????#? 1,3
.???.?.????##??#? 2,7
?#??#?????.?# 1,2,1,1
???#.#???#??.?##?.? 1,7,3
.?#?.#?.#?? 3,1,1
##?##????????? 7,1,1
..?#?###?????? 2,3,1,2
???.??..????..#?#?? 2,1,4,5
.?.?????????? 4,1
.????#?#?.?. 1,3
??#?##???.#??..? 7,2
????#????? 1,6
.#??????##??#?.?#?. 1,10,2
?#??#?.??#???.???#? 5,3,4
.????????#?????. 10,3
??#???#??#?.? 6,1
??.????#????#??? 1,1,4,1,1
??.?.???##.? 1,5,1
??.###????#??## 8,2
???#???????.?????.? 9,1,1,1
.????#??#??#???## 4,1,6
????..##?.?.?..???? 1,1,3,1,3
?.??.?#?#.?#?##??#?? 1,2,4,6,1
.????#???..????# 1,4,1,3
?##?.?.???#?#???.? 2,6,1,1
.??.??#???? 1,3
?#?.?????#????##??? 1,11
?#??##?#????? 1,5,3
.#?#.#.#.??#?#?? 1,1,1,1,7
#?#???#?###?##??#? 3,9,2
???.????##???????? 1,1,7,1
.?###?????.???##?#? 6,7
.#?#.#???#?????##?# 1,1,1,5,1,4
?????????#?#?#??? 1,2,4,1,3
?.???.#?#??? 1,3,1
#?#??#??.#.#???.. 6,1,4
?#????##?#????.? 10,2
????#????? 1,3,1
??.???.?.. 2,1
?.#???#???????? 2,2,2,2
?.?#??????#?. 1,7
.?????.??? 2,2
?#??.??#?? 2,3
?????#???.??.?..??. 1,1
??..???##???.?. 1,7,1
??#?????#. 1,1,4
#.?#.?##.#????? 1,2,2,2,1
??#??#?##.???# 6,2,1,1
??#??????##. 6,2
.#???.##.?. 1,1,2
??.?????#?.??#??#??? 2,1,2,7
?.??#.??.?.?.??.?. 3,1
??#??#.?#.#? 2,2,1,1
?#?????#?.? 3,3,1
??.#?.??#??.#??. 2,1,3
.???#?#??##.? 5,2
..???#????###?#??#? 4,1,3,1,1
?.???#?.????.?????. 4,2
.#??#?????# 1,1,2
.???#????#?? 1,8
#????????.####? 1,1,1,1,4
??##??????????? 9,2
?.??#?#????? 4,2
?..?????.?.#??? 3,1,3
#??.?#???. 2,1,1
?.#??????????? 1,7,1,1
?.???.?#?#? 2,5
???.?#???#??#??? 1,2,8
???#??#?.?#???? 1,1,3,1,1
???.?#?.?????#????## 3,2,2,4,2
##.???.?#??? 2,1,1,3
?.????.?.??.? 3,2
.??.?.?##????.???? 1,6,3
???.?#???? 2,3
?.???????. 1,2,1
.#???????#.???????. 3,2,5
..??.???#?..???#???. 2,2
???.??.#???##?#?? 1,2,1,6
?#?????#?? 2,3
??..??#??##?.?# 2,8,1
????????##??#??.??#? 13,1
???###.?##.??? 5,3,2
??#..??.????.#?. 3,1,4,2
?.?#?.??#? 1,2,4
?#???#?#?.# 8,1
?#?.??.???? 3,1,3
??##????#?? 3,1,2
?????????##??????? 1,12,2
??#??##??????##??#?. 8,7
??.??.???.? 2,1
???..#???????? 1,6
??.???##?? 1,5,1
#?.???.?#.?#???.#?? 1,2,1,1,1,3
??#????.?##?#???#?. 5,9
...#.???????? 1,5
?.?????????#?#????? 1,1,3,6,1
##?.#??.????.??# 2,3,3,3
??##????..??? 7,1
.??#.?.???.?.?#?..?? 3,1
??#????##??.? 1,5
#?????#.?#????.??? 1,4,1,2,1
#??..???.??.#??#? 3,2,1,4
.?#??#?..#?.#? 6,1,1
???##?#?#?.?# 3,4,1
???.#.?..??.#.???# 3,1,1,1,1,2
?.?.#.????? 1,1,2
???.??????#?# 1,2,5
????????.?????.?# 1,2,4,2
???????????#??? 8,1
?..#?.????#???##?# 1,1,10,1
..####??..?? 5,1
???#?###?#?.???? 9,1,1
.??.?????????..? 1,1,6
.?##?.?#.? 2,1
#??#??.??#..?.??.?? 6,2,2,1
??#??#?.?? 5,1
..#####?#???????? 8,4
?????#?#??.???# 6,1,2
?#?#????####????##. 5,1,10
??.??..?##??? 2,3
?.?###...?#?#. 3,3
?????????? 4,1
##.?.?#.??#? 2,2,1,2
??????#.??.#? 6,1
.?#???.??#?? 4,4
????#???#?#?#??#???. 5,12
?#..?.#???.??. 2,4,2
#.??#?#??.?#?#??#. 1,4,1,4,1
#.?..?.?#.?????????# 1,1,1,1,10
?##??##????###???? 3,2,1,5
?#??.#?????.???#?.?? 2,3,5
?.??.????? 2,1
??????##???.?? 2,2,1
??.#???#??#??.??##.? 2,3,5,3,1
?#?##????????? 5,1,1
???##??##???##???.? 14,1
??#.??????..## 2,1,1,2
.?#???.??? 2,3
.#????.???#.??#???. 3,3,5
???.?#?#??#??..??? 1,9,2
???#???????#???#??? 1,4,4,1,2
.?.?????????????#?.? 1,15
??#?#?.?????..? 3,1,3
???.??#?#?# 1,7
.??#??.???#.. 4,1,1
.??????.#????.????. 3,4,2
??#.##.#.?.#??.??##. 2,2,1,1,1,3
#.?.?..????? 1,1,1,2
???.##?.#.?##???#??? 1,3,1,4,2
##.??##??#. 2,3,1
..???#???????.??.? 8,1
#?.??.?????? 1,1,1,1
?###.?..??.#???#? 3,1,1,1,2
?#??????.?? 4,1,1
??.???.?.? 1,1
????????#???????.# 9,1,1,1
???.?#?#?#??? 2,7
#..??.?.?? 1,1,1
.??#????##??.????#? 9,5
??????.???#?? 1,2,1,1
..???..??#??#?#?? 1,9
????.#..##??#? 3,1,6
???????.... 2,1
##.???..?? 2,2,1
##??#???#????????#?. 5,4,5
.??..??#????? 1,1,2,1
.?????.##??? 3,3
?????#??#?##? 3,1,5
.#????.??.??.?????. 1,2,1,2,1,1
???.?.?#????..?##??? 1,1,1,1,3,6
???##?#?????#? 1,2,4,1
???##???????##??? 1,5,8
?#???????##????## 5,1,8
??.#?.?#?? 1,1
?????????????? 1,3,1,4
?#.???#??#?##???# 1,1,1,8
???.????##?. 1,1,1,5
?##..#???.??#. 3,3,1,1
?#?##?.?????.??## 5,1,4
?.???????.?. 1,1
.?.??.??## 1,1,2
??#?????.?# 5,1
?.?#.??????##.?#? 1,1,8,1
?#???.?.???? 4,1
???#?#??##??.?? 1,7,1
???..#????#??#?? 1,10
???????#???????##.# 2,1,2,1,6,1
?.?#??#.???? 1,1,2,1
..??##?.#???#??#??? 1,2,2,3,1,1
???????#?.##?# 8,2,1
?????..??? 2,1,1
#??.#.??#?# 1,1,3
??#?#?#???#.? 6,1
??#????##????? 4,5,2
???#??#??#.?#?##??? 2,7,5,1
.????#???? 4,1
.####?##?#??.#? 9,1
?????.?#?####? 2,8
?.?????#??#???#??? 1,1,1,1,6,1
?#?????#?#????? 3,3,1,3
##??#..?#?.???. 5,2,3
??##?????? 1,4,2
#?.???.?#?#? 1,1,1,1
#.??????????#????#?? 1,2,8
?#?#???#?? 3,1
??#?##??????#.???. 11,1
??..##???? 1,2,1
????#??#?????#???? 2,1,9,1
#???#???.???#??? 1,2,1,1,3
?#??????#.???? 6,1,2
???????????? 1,1,1,2
???.??????.??? 1,1,1,2
?.#???.???##.?.? 3,5
#?????#?##?#??#????# 13,2,2
?.??????.??#? 1,1,2,1
.??.???#??##?#? 1,1,1,6
???????????. 6,1
.????.????#.. 3,4
?.?#?????#?? 4,2
?#?#???????#?#?????? 10,1,1,2,1
???????????? 1,1,5
?????????# 1,5
..?#?###.##.??# 6,2,2
??#????##?#???? 2,3,3
#?#?????.??#?#? 1,2,6
.#??.#.???????.?..?? 1,1,1,7,1,1
????#??##??#?#?#?# 3,1,5,1,1,1
??..???####???????? 1,7,1,1,1
???...#.?? 2,1,1
.?.??#?..#???#?.??#? 3,6,2
?#??????#?? 5,1
?#??..????#?#???? 3,7,1
??.???#?.? 1,2,1
???#??.???#??? 3,1,2
???##???????#?. 1,3,1,3
???#?????#?..#?#? 1,5,3,3
??.?.?#?#??##?#? 1,1,4,4
.###???????#?# 3,8
.#??.?.???#?# 3,1,4
#?.???.???#. 2,1,2
##??#?.??. 2,2,2
#...???#?##?..#? 1,1,1,3,1
?..?#?#?#?.?????? 6,4
#?#????#????#.?.??? 1,9,1,1,2
#..??#.?????#???##? 1,3,3,7
.????.???#??###?? 1,8
?##?#????#? 3,2,2
????.??..#? 1,1
?.????###????? 1,6
??#?#.???#?#?? 4,3
?##?#??#?..???.#?. 8,2,2
?.#????#????#?.#???# 1,2,3,4,2,1
?#?.#????. 2,3
.??##??###???# 10,1
??#??##???...?? 6,1,1
#?????.?..#.? 5,1,1
?????#??????. 2,4,3
????#????? 1,1,2
?.?#???##? 1,1,5
#????.?.?????? 4,1,4
.??????????? 1,3,2
?.??????#??? 1,4,1
?.?#?????? 1,3,1
?.??.###???##????#?. 1,1,3,7
?#??????#.#. 2,1,1,1
.?##?.?????##??## 3,10
????.???#?? 1,2,1
??#????.?#?? 6,2
??.?.#?????.#.?? 2,1
?.??.?#?#?? 2,1,1
?##?.???#? 4,3
?#??.???#?.?? 2,1,4,1
.???????????.# 1,3,4,1
?#???.??#?????.? 1,2,1,6
??#?#?#??.#??. 2,4,1,1
??.?.??#???#?.??.?.? 1,1,6,2,1
#.#?.#???#?#?????#? 1,1,14
.???.#???#??? 2,2,3
##????????#??.?#..? 2,2,2,1,1
?#???#?..?. 1,2,1
??#???#?????#???.?? 3,5,4,1
#??????????#?#?? 1,3,7
?#???#??#?#?? 6,5
??????????? 1,6
#?????????????#? 1,2,6,2
????#??.#.??#. 1,2,1,3
?#????###?.?#.#??. 4,3,2,2
#????#??##????? 1,1,8,1
????.???????#?#? 4,8
?????#??#???#?.?? 6,3,3,1
?????.???..?#.??? 1,1,3,2,1
.?#??#?..???# 4,3
???...###?. 1,4
?#?.?????? 2,3
#?..?????. 2,1,1
????##???? 1,4,2
????????##???#?? 1,1,1,8
??.?..?##?.?#?##?.? 2,1,4,2,3,1
????.???.? 1,2
???????###? 2,5
???.#?.#.?##?? 1,1,1,5
?#????#???.?.? 9,1,1
????.??????##? 1,1,4
.???##.???..#? 5,3,1
?#?##?#?###????????? 5,1,7,1
#..#?.?#????##. 1,1,1,1,3
?.???#???????#???# 4,4,2,3
??.?.??###???????? 1,11
????#?????.??? 2,2,3,1
.?????????####...? 3,6
##??#?##??..??# 10,2
????#?#??.#??? 8,3
#.?????.??#?##?#? 1,1,1,8
#?.??####????.???#? 2,6,1,3
?#?????#.??? 3,2,2
???.##?#.?????????? 3,2,1,1,2,4
??##?##?????..??? 10,1
??.???????#??????? 4,8
###?.???#??? 4,1,2
##????#??#. 4,2,1
.?#?#???.??###?#??# 4,2,1,3,1,1
??????.?..#?.??# 2,2,2,3
.???????#?. 2,5
#?#??????? 4,2,1
.#?.???#??#??????#.? 2,11,1,1
??.????..##??..?### 1,3,2,1,4
???.?????#??? 1,6,1
???.?#??#? 1,4
#??.?????#?. 3,6
??#.#??.??????###? 1,3,8
.???.#????.??? 1,1,5,2
.#??????????? 1,2,2,4
#..?#??#??? 1,8
????#????? 1,1,5
?.???.?????.?????#? 2,4,7
#.?..?##??..??#??? 1,1,4,1,1,1
.???????#????. 2,8
###?..#.??????.?? 4,1,1,1,2
???.??.?### 1,4
???#.?#??.???. 2,4,2
.?#????#?.#???? 1,4,1,1,1
?.??##??#??? 4,5
##????#?#?. 2,4,1
.???#?#??#??#?.#?#? 10,4
.?#?#???.?#.??# 2,1,1,1,1
??.?????#..#??.??? 1,3,1,1,1,1
?...#?#?????????.. 4,3
.?#??????#???#?##.?? 11,4,1
?#???##...?.?????.?? 6,1,3
?#?.?#?#?? 2,1,2
?????##??#.????.???? 8,2,1
.???????#????##???.? 7,1,6,1
??##?????# 5,2
??##..#.?. 4,1,1
????##???? 1,2,2
???#????#?##???#??? 12,1,1
.???????#??.??? 8,3
?#?.##????? 1,7
??????.?.?# 4,2
??#???.#?#?#??? 1,1,1,3,1
???#???#?#???. 1,1,4,1
.???#?????#?..#???.# 4,5,1,1,1
???.??.#???#?. 1,2,1
..????.??##?? 1,6
??#?.?.?#??.??#. 1,1,1,2,3
#????.?#????#???#?. 1,1,1,4,1,1
????????????. 6,1
??.??#??#????.????# 2,7,5
#???????###.?.????. 1,1,6,3
???#?#???#?###? 6,7
????.#?#.??#?.???.? 1,1,3,3,2
#?????.???#???? 5,5
#??#??.#.?????#?. 1,2,1,7
?????#???. 5,2
????????##???.??.? 2,6,1
#?????????? 3,2
.??#????..??????#?? 2,1,7
????????#?##?? 9,2
#?.??????.??? 1,3,1,2
?????.????#?.?? 2,1,1,2,1
#???.??#???? 1,1,3,1
????.??#?. 4,1
???????#.???????? 6,5,1
??.???#??#??????. 1,7,3
???#??##??#????????? 1,13
.#?????????.#???.? 6,1,1,2,1
?????????.. 2,1
??????#????? 1,2,2
#?#????..????#?? 1,4,1,2
?.?.?#????#. 1,1,2,3
#??###??.?#????#???? 8,8
.#.?#????# 1,1,4
?#???#????? 1,6
????.?#???#.?. 2,2,3,1
??.?#?#????? 2,1,3
##???.??.?##????. 2,1,2,4,1
?????#?..?#?#?. 5,3
??#????#????. 10,1
????????????#.?? 1,2,6,1,1
#.?#??.???? 1,2,4
#..???#??? 1,4,1
?????#????#? 4,2
???.##???.. 1,3
?.##?#.?????.?. 4,3,1
??##??.?.??# 4,1,3
?#???#.?.??#??.??. 2,3,1,1,2,1
#??##????#???. 10,1
.??###?#???#?##?.#.? 14,1
?###???????????? 3,1,1,1,2
#?.???????.???#? 2,1,4,4
?#???#???? 1,3,1
????#?#????# 1,5,1
???###????????? 9,1
??????????#????????. 2,10,2
?#.??##???.?? 2,3,2
#?.???.??##? 1,1,1,4
???#???####???????# 11,1,1
##?.#??.?????.?#??#? 3,2,5,1,2
#?.??????###.?#?# 1,9,2,1
??#??##??.#####?. 9,6
?#?.???#?#.?????? 3,5,1,1,1
???##.#???####?.?? 3,9
????.?.###??.?#?? 2,1,3,1,4
.?#??.?####.# 4,5,1
.????.?????#?? 1,3,1
?.##????.???#? 3,1,4
????.###?????.? 2,4,3,1
?.?#.?#?????#??? 1,1,8,1
?#?????????.????? 2,5,2
..#????#?#?..#?????# 1,7,2,2
#????..???#??# 1,1,7
???.??.?#?.?? 3,1,2,1
?#??#?#??##?#??#?. 3,1,11
????#?#?###? 1,8
?#????.???#.? 4,1,1
.?#??????????. 1,2,5
.??#??#??. 2,2
???###?????#???#??#. 7,2,4
??.???.?.????? 2,2
???#??#??# 1,7
???#?.??.????#?##??. 4,2,10
#??.??#??????#???# 2,1,8
????#?????#????#? 1,5,1,4
?.#?##??????? 1,5,1,1
#??.?#??????? 1,6,1
?..?##??######??.?? 1,11,1
??#??#.?#?#???????? 1,4,2,1,2,3
??????????##???.. 2,4
.???????#.#? 1,1,2,2
???????#??##.? 1,1,1,5
???.??#??? 1,5
.#?#.?#?#?# 3,4,1
.?#?.??.?#???? 2,1,2
.?.#?#????#???? 3,3,1
#?????????#..##?#? 1,2,1,1,4
??##??????.?? 4,1,1
.??????#?#? 3,4
?#.?.?..?#?. 2,2
?#??.?#?#??.??????# 2,2,2,1,1,2
?##???##??????..# 8,5,1
??.?#?????????#?? 2,1,9
????.???.. 1,1,1
???#?.???? 3,1
??.??????#???????? 2,6
??#???##???.#?#?. 3,5,4
##???????? 3,2,2
.??#????#?? 2,4
??????.??# 2,2,2
##..#??..??#???. 2,2,6
..?.#?#??.???????? 3,6
##??????#??????? 2,1,1,6
???#??..???? 4,2
.?#?#??..?????.?? 4,3
#??#????.#?? 8,1
.???##.??.?? 4,1
??????#??##?????? 2,13
##???..?.?? 5,1
?.?#?#???..##? 1,6,3
??.????#.?#??# 1,4,1,1
.#?????????#??.??#?. 5,5,2
??.?..#.#??? 1,1,2
??#?.?#??###??? 4,8
??#??#?##???. 2,8
#.#????#???? 1,1,1,1
#???##?#??#??#????? 2,8,1,4
?????????#??## 1,10
??#???#???#??????. 1,15
?###??.??????## 5,3,4
?#???#?.?? 6,1
?.????.???#?###???? 1,8
?##??????..???? 3,1
??##??#..?#?. 5,1
.###?????#?? 6,2,1
??.#??.#?#. 2,1,1
.??.?##???##.#???# 1,8,2,1
?#?#????.##????? 4,3,2,1
?????????.??.? 3,1
##?.???????#???#? 2,10
??#..#???#???#?. 1,1,7
?.#?#..?##?..?? 3,3
??#???????#??#. 1,1,7
#..?.#???#??# 1,1,2,5
.????##???. 1,6
???#???.???#???? 4,5
??##????#. 3,1
??.?#.??.?. 1,1,1
.??.????#??? 1,1,1,1
?.###?.????.? 4,2
???##????.?.?.?.?##? 4,1,1,1,4
???##?#??#?#.# 1,5,3,1
?.?????#?????? 1,1,4,1
..#??##?#?.?#?.? 7,3
#..??#????????????? 1,3,2,4,1
?#.?#??.#?#?????? 1,4,5,1
#???????##..#????? 10,6
..???.#??? 2,2
??#??.?.#??. 2,2
?.?#??.?.?#..?? 3,2
???#?????? 3,3
????????.????..?# 3,2,1,1,1
?????????#?.# 1,1,4,1
#?#.?..?#?.?. 3,2
???.??##????????. 2,9
??#.#.????#.??? 2,1,1,2,2
????.?#.??? 1,2,1
#???????.???.?. 1,1,2,2,1
???.????##????? 2,3,1
?????..??#? 1,2,2
.#?.?#???? 2,1,2
#?????????? 5,2,1
???#?#?..?#????#??.. 6,9
.#???.?????#????. 1,2,4,5
????.??.#???? 2,1,1,1
#?.???##??? 2,1,2
??.?###??.????#?.??? 1,6,4,1,1
#?..?#?##?.? 2,5,1
?????.#????.???# 1,1,4,1,2
#?#??#????#????# 3,2,2,4
???.?????####.##?.? 2,3,5,3
??????..#.? 1,2,1
#????.#???.#?##?? 2,2,5
.??.?#?###?????#??? 1,13
?.??#.??????#?? 3,7
#???.#.????#?.#?#??? 1,1,1,4,3,2
????.#????. 3,1,1
?.???????#??#??.?? 1,4,3,1
#?.#???..??? 2,1,1,2
.?#?#????.? 2,5
??##??#??# 7,1
????##.??##?? 1,2,3
???????????#??? 1,3,2,1,1
??????##?..#?#??.?.? 9,1,1,1,1,1
.???.???.?? 1,1
#?.??.?#???????..? 1,1,1,4,1,1
?.?#?#?.???#?. 5,3
?????????.??..#?.? 5,2,1,1
?#???.???##?????? 3,10
?##??.#?????#.??#? 4,4,2,1,2
.#.?.??.?? 1,1,1
???#?.???.?.???? 2,3,2
?#.?#??.?? 1,1,1
#??##.?.?#?#?????# 1,2,10
?.????###????.? 8,1
?##???????? 4,4
??.?.#?.?#?.###???#? 2,1,2,1,8
?##?#??????.?.??#?# 6,3,1,1,3
?.?.#??##???#?#? 1,1,5,1,1
????.???#?#???#? 3,6,1
..#??###??????### 6,7
.???????.??#. 1,1
?#????????#???#??? 2,2,4,2
.#??#?#?????. 1,2,2,2
??..?.?????##????. 2,1,3,2,1,1
??.?#.?.#?? 1,1,2
?????????? 7,1
#??????.?..??????? 6,1,1,1,1
??.#???#?????.?????? 1,8,1,1,1
.#.#????????. 1,1,5
???#?#..##??## 4,1,3,2
.#???#????????.# 1,9,1,1
?##???????? 3,1
??#????#.#? 1,1,1
.??###.?#???#??#???? 5,3,6
.???#?#???####??#?? 7,8
???.?.??????????? 1,10
#.??.?.#?..# 1,2,1,1
????#.??#... 4,2
#?????..??.? 1,1,1,1
#?????##???.? 1,1,3,1
????????.??? 1,1,1,3
.??####?.?#??#??? 4,5
?#?#???..? 1,2,1
??#???????###??. 5,7
?.?#?#?##?.?#?##. 7,4
###.##????#???.?# 3,2,2,1,2
..??#?..?#??.##?#?## 1,1,3,2,1,2
?#????????????##???? 1,1,3,1,6,2
???.?#??.??? 1,3,1
???##???????.??##?? 4,1,2,4
.?.?.?#??## 1,5
?.##??????.???.? 6,1,2
.?????.?###? 1,5
????.?????. 2,1,1
#?????.?????.#??.?#? 1,1,1,1,2,3
????????.#?.. 6,2
?.#???.????.?.??#? 1,1,2,1,1,2
?#????#?#????#?..?. 14,1
#..??#?#???? 1,1,6
?.?#?#?????? 4,1
????##?#?.#?. 5,1
?????###??.#?#??.? 1,7,5
#??????.???? 1,1,1,3
??#????????. 1,5
...#??#..????. 4,1,2
?#??#??#?.? 5,2,1
??#??.?#??????????. 3,6,3
?.#?#?????? 5,2
???#???##?.??. 8,2
..?.#??????.? 1,1,1
???#.##??#? 3,5
?.????#???????.#? 1,1,4,1,1
?#?????.??? 2,2,2
?.?#?.?????.?. 1,1,1,2
?????.??????.??????? 3,2,1,1,1,2
#?????#??..#??#??. 9,5
..#?????.. 1,1,1
?..???#??#?????#?#?? 5,4
##??????##????.?? 13,1
????##???#????. 1,6,1
???.#?#??????# 1,4,2
.???.??### 2,3
?.#??????####?#??? 1,1,1,1,7,1
?.?#.?.?????##.? 2,1,1,3
??#??.###?? 4,3
..????#???????? 2,7
.??????????. 2,1,1,1
?#.???#?#????? 2,1,4,2
#?##???.?#.?.#. 5,1,2,1,1
#.?#.?#?#? 1,1,4
??.???##?. 1,1,3
????#.???#?## 2,1,2,4
???#?#?#??##??#??#? 1,15
#?.????#?????.????#? 2,1,6,1,2
.?????.#??? 1,1,4
#?.???###?#???. 1,1,8
?.??#?#????????# 1,5,1,1,2
.????.???##?#.??#?? 1,1,7,3
.####.#??# 4,1,2
?#?????.#.?.?## 2,1,1,1,3
??#?.??????# 1,1,2,3
????.#??????? 4,5
#..???.?.???##.??? 1,3,1,1,2,1
.?#??..?.?##?????#? 4,1,2,2,1
??#.??.?.??# 2,1,1,1
#???#???????? 9,2
.?#.????#???? 1,2,2
#.??.?#.???.??? 1,1,1,2,1
?????##????.?.?? 10,1,1
?????#?.#???.?????? 6,1,1,1,1
?#???????. 1,1,1
????#??????##???.? 2,1,2,2,1,1
???.?##??????#???.? 3,3,7
?.#?.???.#?? 2,1
?..??#????.?.?..##?? 7,3
?#?#?#?????????# 4,1,3,2,1
?.#.#?.?????? 1,1,1,2
#??.?##??..?????.? 3,4,2,1
????.#????##??..?#? 3,1,3,1,2
??#?..???#?? 3,1,1
???..?#???### 3,1,1,3
.??????..???## 5,3
##??????.???? 4,1
?#????##?#?.???? 1,5,3
..?##?#??????##.??#? 9,2,1
..#??.?.??. 1,1
.???.?#???????#??? 1,5,1,1,1
.???.??#?? 3,3
??#?##??#?.?#? 10,2
?.???.???.?? 1,1,2,1
????.????# 2,2,1
??????.?????? 2,3,1
??.?#..#?#??.?# 2,1,5,1
?????#?.?#???..???? 2,3,2,1,1,1
??.??####?##.??. 1,8
?#????##?##.##??? 2,7,5
.#???#..???#?#???#?? 2,1,11
?##??#?.????? 6,2,1
#???.?#??..??? 1,4,2
???????###? 4,3
??.??#??## 1,1,5
?.?#??.?#?#???. 2,6
#??.?????#?##?#??? 1,1,1,1,9
?#??#.??#??####??? 5,9,1
#??.??????. 3,5
#???#??#.??#? 1,6,1
.?.???##??.?#????. 5,3,1
?#?##??????##???#??. 1,10,3
.?????????.?? 1,1,2,1
?.???#???. 3,1
????????#?#?? 5,4
?.?????##.????#????? 1,1,2,1,5
??#???#?#??.. 1,1,5
??##??##?.#...??.?? 4,4,1,1,1
?#??##?##?##?#???. 11,2,1
?..????#???. 1,1,3
????#???????. 3,2
?###.?#??.???#.?#? 4,2,1,1,1,3
?#?#?????.?#?? 7,1,1,1
???????#???. 1,2,1
#?##??#?..?#?.?# 7,2,2
???#.#??.??#?. 3,3,4
.???..##?? 2,3
#.?.?.?..??##?? 1,1,1,5
.?#??#??#? 2,1,1
?#?##??.#??? 2,3,1,1
#???#????.????????#? 6,1,3,4
.#..????????#? 1,1,1,3
????.#??..??.? 1,2
#??#..?#??#????? 4,1,5
#?#????.????#.???. 7,1,1,1,1
.?????????? 1,2,1
????#?.??? 1,3,1
"""

# Solution

In [11]:
game = Game.from_str(value=puzzle_str, part_2=False)

In [12]:
game.solution_part_1()

100%|██████████| 1000/1000 [00:46<00:00, 21.59it/s]


6958

In [13]:
from functools import cache
from re import match

print(sum((g:=lambda m,d: not d if not m else (m[0]!='#' and g(m[1:],d))+(d and match(r'[#?]{%d}[.?]'%d[0],m) and g(m[d[0]+1:],d[1:]) or 0))(s[0]+'.',(*map(int,s[1].split(',')),)) for s in map(str.split, puzzle_str.strip())))

# print(sum((g:=cache(lambda m,d: not d if not m else (m[0]!='#' and g(m[1:],d))+(d and match(r'[#?]{%d}[.?]'%d[0],m) and g(m[d[0]+1:],d[1:]) or 0)))('?'.join([s[0]]*5)+'.',(*map(int,s[1].split(',')),)*5) for s in map(str.split,open(0))))

ValueError: IPython won't let you open fd=0 by default as it is likely to crash IPython. If you know what you are doing, you can use builtins' open.